<a href="https://colab.research.google.com/github/Madhav2204/LGMVIP-DataScience/blob/main/Task_10_ML_Facial_recognition_to_detect_mood_and_suggest_songs_accordingly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Author : Madhav Shrivastava

Task-10 : ML Facial recognition to detect mood and suggest songs accordingly 

Dataset Link: https://www.kaggle.com/msambare/fer2013

In [ ]:
import numpy as np
import pandas as pd
import os
import datetime
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import regularizers
from keras.utils.vis_utils import plot_model
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator, load_img,img_to_array
from keras.layers import Conv2D, Dense, BatchNormalization, Activation, Dropout, MaxPooling2D, Flatten
from keras.callbacks import ModelCheckpoint, CSVLogger, TensorBoard, EarlyStopping, ReduceLROnPlateau
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.datasets import make_classification
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [ ]:
print(os.listdir("train"))

FileNotFoundError: ignored

In [ ]:
def load_dataset(directory):
  x, y = [],[]
  i=1
  for subdir in listdir(directory):
    path = directory + subdir + '/'
    faces = load_faces(path)
    labels = [subdir for _ in range(len(faces))]
    print("%d There are %d images in the class %s:"%(i,len(faces),subdir))
    x.extend(faces)
    y.extend(labels)
    i=i+1
  return asarray(x),asarray(y)  


trainX,trainY = load_dataset('/content/drive/MyDrive/Indian-celebrities/')
print(trainX.shape,trainY.shape)
savez_compressed('/content/drive/MyDrive/PROJECTS/face recog + mood/Indian-celeb-dataset.npz',trainX,trainY)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggleSPOTIFY.csv to kaggleSPOTIFY.csv


In [ ]:
data = pd.read_csv('/content/drive/My Drive/projects/train.csv')
test_data = pd.read_csv('/content/drive/My Drive/projects/test.csv')

In [ ]:
spotify_df.dropna(subset=['consolidates_genre_lists'], inplace=True)

In [ ]:
spotify_df.isna().sum()

Unnamed: 0                  0
valence                     0
year                        0
acousticness                0
artists                     0
danceability                0
duration_ms                 0
energy                      0
explicit                    0
id                          0
instrumentalness            0
key                         0
liveness                    0
loudness                    0
mode                        0
name                        0
popularity                  0
release_date                0
speechiness                 0
tempo                       0
artists_upd_v1              0
artists_upd_v2              0
artists_upd                 0
artists_song                0
consolidates_genre_lists    0
dtype: int64

In [ ]:
spotify_df.head(3)

,Unnamed: 0,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,consolidates_genre_lists
0,0,0.177,1989,0.568,['조정현'],0.447,237688,0.2150,0,2ghebdwe2pNXT4eL34T7pW,0.000001,10,0.0649,-16.478,1,그아픔까지사랑한거야,31,1989-06-15,0.0272,71.979,['조정현'],[],['조정현'],조정현그아픔까지사랑한거야,['classic_korean_pop']
1,1,0.352,1992,0.381,['黑豹'],0.353,316160,0.6860,0,3KIuCzckjdeeVuswPo20mC,0.000000,11,0.0568,-9.103,1,DON'T BREAK MY HEART,35,1992-12-22,0.0395,200.341,['黑豹'],[],['黑豹'],黑豹DON'T BREAK MY HEART,"['chinese_indie', 'chinese_indie_rock']"
2,2,0.458,1963,0.987,['黃國隆'],0.241,193480,0.0437,0,4prhqrLXYMjHJ6vpRAlasx,0.000453,5,0.2650,-24.571,1,藝旦調,23,1963-05-28,0.0443,85.936,['黃國隆'],[],['黃國隆'],黃國隆藝旦調,[]


In [ ]:
mood_prep = spotify_df[['duration_ms', 'danceability', 'acousticness', 'energy', 'instrumentalness',
       'liveness', 'valence', 'loudness', 'speechiness', 'tempo']]

In [ ]:
col_features = mood_prep.columns[:]
col_features

Index(['duration_ms', 'danceability', 'acousticness', 'energy',
       'instrumentalness', 'liveness', 'valence', 'loudness', 'speechiness',
       'tempo'],
      dtype='object')

In [ ]:
mood_trans = MinMaxScaler().fit_transform(mood_prep[col_features])
mood_trans_np = np.array(mood_prep[col_features])

In [ ]:
mood_trans_np[10011]

array([ 3.37093e+05,  4.26000e-01,  1.01000e-01,  6.46000e-01,
        0.00000e+00,  3.58000e-01,  8.21000e-01, -1.10330e+01,
        1.32000e-01,  2.05703e+02])

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving data_moods.csv to data_moods.csv


In [ ]:
df = pd.read_csv('data_moods.csv')

In [ ]:
cl_features = df.columns[6:-3]
X= MinMaxScaler().fit_transform(df[cl_features])
X2 = np.array(df[cl_features])
Y = df['mood']

In [ ]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_y = encoder.transform(Y)


dummy_y = utils.to_categorical(encoded_y)

X_train,X_test,Y_train,Y_test = train_test_split(X,encoded_y,test_size=0.2,random_state=15)

target = pd.DataFrame({'mood':df['mood'].tolist(),'encode':encoded_y}).drop_duplicates().sort_values(['encode'],ascending=True)
target

,mood,encode
5,Calm,0
4,Energetic,1
0,Happy,2
1,Sad,3


In [ ]:
def base_model():
    model = Sequential()
    model.add(Dense(8,input_dim=10,activation='relu'))
    model.add(Dense(4,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',
                 metrics=['accuracy'])
    return model

In [ ]:
#Configure the model
estimator = KerasClassifier(build_fn=base_model,epochs=300,batch_size=200,verbose=0)

In [ ]:
#Evaluate the model using KFold cross validation
kfold = KFold(n_splits=10,shuffle=True)
results = cross_val_score(estimator,X,encoded_y,cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100,results.std()*100))

Baseline: 77.70% (4.97%)


In [ ]:
estimator.fit(X_train,Y_train)
y_preds = estimator.predict(X_test)

In [ ]:
y_preds

array([1, 0, 0, 1, 1, 2, 0, 1, 2, 1, 1, 1, 1, 0, 0, 3, 3, 1, 3, 2, 0, 0,
       3, 0, 1, 3, 0, 1, 3, 1, 1, 1, 0, 1, 3, 0, 2, 1, 0, 3, 1, 0, 0, 3,
       3, 3, 0, 2, 1, 0, 2, 1, 3, 3, 1, 3, 1, 1, 1, 2, 1, 0, 1, 0, 3, 1,
       2, 2, 0, 3, 1, 3, 3, 0, 0, 0, 3, 3, 1, 0, 3, 2, 1, 2, 0, 1, 3, 3,
       0, 1, 1, 0, 1, 2, 3, 1, 1, 0, 3, 1, 2, 0, 0, 2, 3, 2, 2, 0, 1, 2,
       3, 3, 0, 0, 2, 1, 2, 3, 3, 0, 3, 0, 2, 0, 1, 0, 0, 0, 0, 1, 3, 3,
       1, 1, 1, 3, 3, 2])

In [ ]:
#Join the model and the scaler in a Pipeline
pip = Pipeline([('minmaxscaler',MinMaxScaler()),('keras',KerasClassifier(build_fn=base_model,epochs=300, batch_size=200,verbose=0))])
#Fit the Pipeline
pip.fit(X2,encoded_y)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('keras',
                 <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fd5195e6810>)])

In [ ]:
def predict_mood(preds):
    
#     pipe = Pipeline([('minmaxscaler',MinMaxScaler()),('keras',KerasClassifier(build_fn=base_model,epochs=300, batch_size=200,verbose=0))])
#     #Fit the Pipeline
#     pipe.fit(X2,encoded_y)

    preds_features = np.array(preds[:]).reshape(-1,1).T

    #Predict the features of the song
    results = pip.predict(preds_features)

    mood = np.array(target['mood'][target['encode']==int(results)])

    return str(mood[0])
    #print(f"{name_song} by {artist} is a {mood[0].upper()} song")

In [ ]:
res = []

for i in range(len(mood_trans_np)):
  res.append(predict_mood(mood_trans_np[i]))

KeyboardInterrupt: ignored

In [ ]:
spotify_df.shape

In [ ]:
print(len(res))

In [ ]:
spotify_df['Mood'] = np.resize(res,len(spotify_df))

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
spotify_df.to_csv('kaggleMusicMoodFinal.csv')

In [ ]:
res.count("Sad")

In [ ]:
res.count("Happy")

In [ ]:
res.count("Energetic")

In [ ]:
res.count("Calm")

In [ ]:
spotify_df.shape